# This notebook aims to find the names of candidate persons in text that has been previously obtained from web scraping.

# We have a data table that contains the text of the news articles and another that contains the names of the candidates. The goal is to identify if any candidate person is mentioned in the text.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 4.3 MB/s eta 0:00:00


In [ ]:
import os
os.listdir('drive')

['Shareddrives',
 '.shortcut-targets-by-id',
 'MyDrive',
 '.file-revisions-by-id',
 '.Trash-0']

In [ ]:
import pandas as pd
import re
from unidecode import unidecode

In [ ]:
# Leer el csv de los municipes
# Eliminar duplicados

municipes = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/8_otros/candidaturas_lista.csv')
municipes = municipes.drop_duplicates()
municipes = municipes.dropna(subset=['persona_propietaria'])

municipes = municipes[~municipes['persona_propietaria'].str.contains("Persona con discapacidad", case=False, na=False)]


In [ ]:
municipes.info()
#6077 entries, 0 to 6647

In [ ]:
def limpiar_nombre(nombre):
    # Usar una expresión regular para reemplazar "DE ", "LA ", y "LOS " por cadenas vacías
    nombre_limpio = re.sub(r'\b(DE|LA|LOS)\s+', '', nombre)
    return nombre_limpio


In [ ]:
municipes['persona_propietaria_regex'] = municipes['persona_propietaria'].apply(limpiar_nombre)

In [ ]:
def generar_regex_nombre(nombre_completo):
    """
    Genera una expresión regular con varias posibilidades de búsqueda para un nombre dado.
    Incluye el nombre completo, la primera y segunda parte del nombre con el primer apellido,
    el primer nombre con cada uno de los apellidos, y cada uno de los apellidos por separado.
    """
    partes = nombre_completo.split()

    # Asumimos que el formato es "PrimerNombre SegundoNombre ApellidoPaterno ApellidoMaterno"
    # y generamos las combinaciones mencionadas.
    combinaciones = []

    if len(partes) >= 4:
        primer_nombre, segundo_nombre, apellido_paterno, apellido_materno = partes[0], partes[1], partes[2], partes[3]
        # Agregando combinaciones para nombre completo, nombre y apellidos, y apellidos individuales
        combinaciones.extend([
            f"{primer_nombre} {segundo_nombre} {apellido_paterno} {apellido_materno}",
            f"{primer_nombre} {segundo_nombre}",
            f"{primer_nombre} {apellido_paterno}",
            f"{primer_nombre} {apellido_materno}",
            f"{apellido_paterno}",
            f"{apellido_materno}"
        ])
    elif len(partes) == 3:
        primer_nombre, segundo_nombre, apellido_paterno = partes[0], partes[1], partes[2]
        combinaciones.extend([
            f"{primer_nombre} {segundo_nombre} {apellido_paterno}",
            f"{primer_nombre} {segundo_nombre}",
            f"{primer_nombre} {apellido_paterno}",
            f"{apellido_paterno}"
        ])
    elif len(partes) == 2:
        primer_nombre, apellido_paterno = partes[0], partes[1]
        combinaciones.extend([
            f"{primer_nombre} {apellido_paterno}",
            f"{apellido_paterno}"
        ])
    else: # Para nombres que no se ajusten a estos patrones, retornamos el nombre tal cual.
        combinaciones.append(nombre_completo)

    # Unimos las combinaciones con '|', que en una regex significa "OR"
    regex = '|'.join(combinaciones)
    return regex

In [ ]:
# Aplicar función generar_regex_nombre
municipes['lista_regex'] = municipes['persona_propietaria_regex'].apply(generar_regex_nombre)

In [ ]:
# Leer datos
texto_df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/data/2_outputs/1_prensa_digital/portales_digitales_reporte5.xlsx')
def limpiar_texto(texto):
    texto = re.sub(r'\s+', ' ', texto)  # Convierte espacios múltiples en uno solo
    texto = re.sub(r'[^\w\s]', '', texto)  # Elimina puntuación
    texto = unidecode(texto)  # Elimina acentos
    texto = texto.lower()  # Opcional: Convierte a minúsculas
    return texto

# Aplicar la función de limpieza a cada fila de la columna 'texto'
texto_df['texto_limpio'] = texto_df['texto'].apply(limpiar_texto)

/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [ ]:
import pandas as pd
import re

# Suponiendo que municipes ya tiene una columna 'lista_regex' con las expresiones regulares
# Y que texto_df tiene una columna 'texto' y potencialmente otras columnas.

# Preparar una lista para recoger los resultados
resultados_list = []

# Obtener el total de filas en municipes para imprimir el progreso
total_personas = municipes.shape[0]

# Iterar sobre cada persona en municipes
for index, row in municipes.iterrows():
    # Mostrar el progreso de la iteración sobre municipes
    print(f"Procesando {index + 1}/{total_personas} personas...")

    # Dividir la 'lista_regex' en sus componentes individuales
    regexes = row['lista_regex'].split('|')

    # Iterar sobre cada texto en texto_df
    for texto_index, texto_row in texto_df.iterrows():
        coincidencias = 0  # Contador de coincidencias para este texto
        for regex in regexes:
            # Compilar y buscar cada regex individual, considerando case-insensitive
            if re.search(regex, texto_row['texto_limpio'], re.IGNORECASE):
                coincidencias += 1
                if coincidencias >= 3:  # Si hay al menos 3 coincidencias, procesar
                    # Si encuentra suficientes coincidencias, combinar info de row y texto_row
                    nueva_fila = {**row.to_dict(), **texto_row.to_dict()}

                    # Agregar el diccionario combinado a la lista de resultados y salir del bucle de regexes
                    resultados_list.append(nueva_fila)
                    break  # Salir del bucle de regex, ya que encontramos al menos 2 coincidencias

# Convertir la lista de resultados en un DataFrame
resultados = pd.DataFrame(resultados_list)

# Verificar los primeros resultados
print("Proceso completado.")
print(resultados.head())
resultados.to_csv('/content/drive/MyDrive/Colab Notebooks/data/7_polaridad/semana_5/municipes_textos2.csv',index=False)

Se han truncado las últimas 5000 líneas del flujo de salida.
Procesando 1138/6076 personas...
Procesando 1139/6076 personas...
Procesando 1140/6076 personas...
Procesando 1141/6076 personas...
Procesando 1142/6076 personas...
Procesando 1143/6076 personas...
Procesando 1144/6076 personas...
Procesando 1145/6076 personas...
Procesando 1146/6076 personas...
Procesando 1147/6076 personas...
Procesando 1148/6076 personas...
Procesando 1149/6076 personas...
Procesando 1150/6076 personas...
Procesando 1151/6076 personas...
Procesando 1152/6076 personas...
Procesando 1153/6076 personas...
Procesando 1154/6076 personas...
Procesando 1155/6076 personas...
Procesando 1156/6076 personas...
Procesando 1157/6076 personas...
Procesando 1158/6076 personas...
Procesando 1159/6076 personas...
Procesando 1160/6076 personas...
Procesando 1161/6076 personas...
Procesando 1162/6076 personas...
Procesando 1163/6076 personas...
Procesando 1164/6076 personas...
Procesando 1165/6076 personas...
Procesando 1166

In [ ]:
resultados['persona_propietaria'].value_counts()

persona_propietaria
JOSE MARIA MARTINEZ MARTINEZ       25
JUAN JOSE FRANGIE SAADE            20
JAIME GONZALEZ GONZALEZ            14
SERGIO ARMANDO CHAVEZ DAVALOS      13
RAMON ALEJANDRO GONZALEZ LEON      12
                                   ..
ARTURO SALCEDO IBARROLA             1
MARIA ELIDIA FLORES RAMIREZ         1
ALEJANDRA NOEMI GONZALEZ ROBLES     1
JOSE PABLO RUIZ BERNAL              1
JESUS ALBERTO MORENO LOPEZ          1
Name: count, Length: 94, dtype: int64

In [ ]:
resultados.to_csv('/content/drive/MyDrive/Colab Notebooks/data/7_polaridad/semana_5/municipes_textos.csv',index=False)